# here we use tow classfier model after pre proccesing

In [26]:
import pandas as pd
import numpy as np
import json
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import sys
from sklearn.neighbors import KNeighborsClassifier

import pickle


%matplotlib inline
rand_seed = 0  # random state for reproducibility
np.random.seed(rand_seed)


In [27]:
data = pd.read_excel('data/datasix.xlsx')
data = data.dropna()
data.head()
data.shape
print(data['label'].unique())



['سلبي' 'محايد' 'ايجابي']


In [28]:
data

,comment,label,lemma,clean
0,النت كعب عندنا شديد يا ناس سوداني يعني الزوال...,سلبي,نت كعب عندنا شديد ناسي سوداني عني زوال حول شنا,النت كعب عندنا شديد ناس سوداني يعني الزوال يحو...
1,بس من السحب متين,محايد,سحب متين,السحب متين
2,اول حاجه صلحو خدمتكم سءيه جدا,سلبي,اول حاجه صلح خدمه هو سءيه جد,اول حاجه صلحو خدمتكم سءيه جدا
3,أفضل شبكة علي الإطلاق والشئ البميزكم اكثر واعج...,ايجابي,افضل شبك هو علي اطلاق و بميزم اكثر و اعجبني نت...,افضل شبكه علي الاطلاق والشء البميزكم اكثر واعج...
4,اجهزو لي المقاطعة وشريحتكم دي ح اجيبها ليكم في...,سلبي,اجهز مقاطع هو و شريحه هو دي اجيب هو لي هو مكتب...,اجهزو المقاطعه وشريحتكم دي اجيبها ليكم مكتبكم ...
...,...,...,...,...
6177,الواي فاي كم الاشتراك بتاعو ومميزاتو من الداتا,محايد,واي ف اي اشتراك ب تاعو و مميز داتا,الواي فاي الاشتراك بتاعو وميزاتو الداتا
6178,اقوي شبكه في السودان,ايجابي,اقوي شبكه سودان,اقوي شبكه السودان
6179,اول حاجه غيرو طريقه عملكم لانها اكل مال الناس ...,سلبي,اول حاجه غيرو طريق هو عمل هو لان هو اكل مال ان...,اول حاجه غيرو طريقه عملكم لانها اكل مال الناس ...
6180,سوداني الاقوي والافضل,ايجابي,سوداني اقوي وافل,سوداني الاقوي والافضل


In [29]:
def random_split(data, features, output, fraction, seed=0):
    X_train, X_test, y_train, y_test = train_test_split(data[features],
                                                        data[output],
                                                        stratify = data[output],
                                                        random_state=seed,
                                                        train_size=fraction
                                                       )
    train_data = pd.DataFrame(data=X_train, columns=features)
    train_data[output] = y_train
    test_data = pd.DataFrame(data=X_test, columns=features)
    test_data[output] = y_test
    
    return train_data, test_data

# using comments after lemmatization

In [30]:
train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
  # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
features.remove('clean')
print('output:', output)
print('features:', features)

4train_data, test_data = random_split(data, features, output, train_fraction, rand_seed)

print(len(train_data))
print(len(test_data))


print(len(train_data)+len(test_data))
print(len(data))

output: label
features: ['lemma']
4928
1233
6161
6161


In [7]:

# TF IDF


vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, max_df=0.5, stop_words=None, use_idf=True)
train_data_features = vectorizer.fit_transform(train_data['lemma'])
test_data_features = vectorizer.transform(test_data['lemma'])

#SHAPE

train_data_features.shape, test_data_features.shape

#FUNCTION FOR MODEL TRAIN



((4928, 27306), (1233, 27306))

In [31]:
def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    print('-'*100+str(clf)[0:15])
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='result_'+str(clf)[0:15]+'.xlsx'
    data.to_excel(filename)
    
    
    sentiment_fit=clf.fit(train_features,train_labels)
    y_pred=sentiment_fit.predict(test_features)
    print(classification_report(test_labels,y_pred,target_names=('posative','negative','netural')))


In [9]:
#LOGASTIC REGRESSION



logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



# MultinomialNB

mnb = MultinomialNB()

train_n_test_classifier(mnb,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)
# S V M

svm = SVC(kernel='linear', probability=True, random_state=rand_seed)

train_n_test_classifier(svm,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


# MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)
train_n_test_classifier(mlp,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

# KNN

knn = KNeighborsClassifier(n_neighbors=3)

train_n_test_classifier(knn, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

#TREE
dtree = DecisionTreeClassifier(random_state=0)
train_n_test_classifier(dtree, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


----------------------------------------------------------------------------------------------------LogisticRegress
accuracy_score Score on training data:
0.9232954545454546
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7639902676399026
f1_score  on test data:
0.7093281731043968
              precision    recall  f1-score   support

    posative       0.89      0.78      0.83       351
    negative       0.73      0.93      0.82       610
     netural       0.70      0.37      0.48       272

    accuracy                           0.76      1233
   macro avg       0.77      0.69      0.71      1233
weighted avg       0.77      0.76      0.75      1233

----------------------------------------------------------------------------------------------------MultinomialNB()
accuracy_score Score on training data:
0.8609983766233766
__________________________________________________

In [10]:


estimator = []


estimator.append(('LR', LogisticRegression(solver ='lbfgs',  multi_class ='multinomial',  max_iter = 200)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('mnb',MultinomialNB()))
estimator.append(('mlp',MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=3)))     


vot_hard = VotingClassifier(estimators = estimator, voting ='hard')

train_n_test_classifier(vot_hard, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

train_n_test_classifier(vot_soft, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Gradientclf = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)

train_n_test_classifier(Gradientclf,train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

adaClf = AdaBoostClassifier(n_estimators=100, random_state=0)

train_n_test_classifier(adaClf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

rf = RandomForestClassifier(n_estimators=100, random_state=rand_seed)

train_n_test_classifier(rf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Iteration 1, loss = 1.19849107
Iteration 2, loss = 1.02771925
Iteration 3, loss = 0.87180472
Iteration 4, loss = 0.66480395
Iteration 5, loss = 0.48850343
Iteration 6, loss = 0.37948538
Iteration 7, loss = 0.28368160
Iteration 8, loss = 0.18548513
Iteration 9, loss = 0.11130360
Iteration 10, loss = 0.06898042
Iteration 11, loss = 0.04938989
Iteration 12, loss = 0.04003324
Iteration 13, loss = 0.03607216
Iteration 14, loss = 0.03262181
Iteration 15, loss = 0.03210651
Iteration 16, loss = 0.03172415
Iteration 17, loss = 0.02921231
Iteration 18, loss = 0.03024626
Iteration 19, loss = 0.02853189
Iteration 20, loss = 0.02819473
Iteration 21, loss = 0.02873557
Iteration 22, loss = 0.03058775
Iteration 23, loss = 0.02777457
Iteration 24, loss = 0.02871577
Iteration 25, loss = 0.02794708
Iteration 26, loss = 0.02715379
Iteration 27, loss = 0.02683352
Iteration 28, loss = 0.02878415
Training loss did not improve more than tol=0.001000 for 10 consecutive epochs. Stopping.
-----------------------

In [32]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

# bow


vectorizer = CountVectorizer(ngram_range=(1, 2))
train_data_features = vectorizer.fit_transform(train_data['lemma'])
test_data_features = vectorizer.transform(test_data['lemma'])
#SHAPE

train_data_features.shape,  test_data_features.shape


((4928, 27306), (1233, 27306))

In [33]:
#LOGASTIC REGRESSION



logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



# MultinomialNB

mnb = MultinomialNB()

train_n_test_classifier(mnb,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)
# S V M

svm = SVC(kernel='linear', probability=True, random_state=rand_seed)

train_n_test_classifier(svm,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


# MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)
train_n_test_classifier(mlp,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

# KNN

knn = KNeighborsClassifier(n_neighbors=3)

train_n_test_classifier(knn, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

#TREE
dtree = DecisionTreeClassifier(random_state=0)
train_n_test_classifier(dtree, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


estimator = []


estimator.append(('LR', LogisticRegression(solver ='lbfgs',  multi_class ='multinomial',  max_iter = 200)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('mnb',MultinomialNB()))
estimator.append(('mlp',MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=3)))     


vot_hard = VotingClassifier(estimators = estimator, voting ='hard')

train_n_test_classifier(vot_hard, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

train_n_test_classifier(vot_soft, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Gradientclf = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)

train_n_test_classifier(Gradientclf,train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

adaClf = AdaBoostClassifier(n_estimators=100, random_state=0)

train_n_test_classifier(adaClf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

rf = RandomForestClassifier(n_estimators=100, random_state=rand_seed)

train_n_test_classifier(rf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


C:\Users\rabab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


----------------------------------------------------------------------------------------------------LogisticRegress
accuracy_score Score on training data:
0.981737012987013
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7599351175993512
f1_score  on test data:
0.7196339464699925


C:\Users\rabab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    posative       0.85      0.81      0.83       351
    negative       0.77      0.86      0.81       610
     netural       0.58      0.46      0.51       272

    accuracy                           0.76      1233
   macro avg       0.74      0.71      0.72      1233
weighted avg       0.75      0.76      0.75      1233

----------------------------------------------------------------------------------------------------MultinomialNB()
accuracy_score Score on training data:
0.932426948051948
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7639902676399026
f1_score  on test data:
0.6958005867378915
              precision    recall  f1-score   support

    posative       0.88      0.80      0.84       351
    negative       0.72      0.95      0.82       610
     netural       0.77      0.30      0.43       272

    acc

Iteration 15, loss = 0.03491901
Iteration 16, loss = 0.03456886
Iteration 17, loss = 0.03191503
Iteration 18, loss = 0.03211688
Iteration 19, loss = 0.03063822
Iteration 20, loss = 0.03002186
Iteration 21, loss = 0.03077645
Iteration 22, loss = 0.03133205
Iteration 23, loss = 0.02948324
Iteration 24, loss = 0.03149737
Iteration 25, loss = 0.03037153
Iteration 26, loss = 0.02891448
Iteration 27, loss = 0.02898964
Iteration 28, loss = 0.02956390
Iteration 29, loss = 0.02895896
Iteration 30, loss = 0.02809926
Training loss did not improve more than tol=0.001000 for 10 consecutive epochs. Stopping.
              precision    recall  f1-score   support

    posative       0.86      0.81      0.83       351
    negative       0.75      0.91      0.82       610
     netural       0.67      0.40      0.50       272

    accuracy                           0.77      1233
   macro avg       0.76      0.71      0.72      1233
weighted avg       0.76      0.77      0.75      1233

Iteration 1, loss

# comments before applying lemmatization

In [18]:
data = pd.read_excel('data/datasix.xlsx')
data = data.dropna()
data.head()
data.shape
print(data['label'].unique())



['سلبي' 'محايد' 'ايجابي']


In [19]:
train_fraction = .80 # use this to split data into training (80%), and tmp (20%)
  # use this to split the tmp data into validation (50%), and 
                     # testing (50%) which means that the validation will be 10% of the original data as well as the

output = 'label' # output label column
features = data.columns.tolist() # the features columns
features.remove(output)
features.remove('comment')
features.remove('lemma')
print('output:', output)
print('features:', features)

train_data, test_data = random_split(data, features, output, train_fraction, rand_seed)

print(len(train_data))
print(len(test_data))


print(len(train_data)+len(test_data))
print(len(data))

output: label
features: ['clean']
4928
1233
6161
6161


In [20]:

# TF IDF


vectorizer = TfidfVectorizer(ngram_range=(1, 2), sublinear_tf=True, max_df=0.5, stop_words=None, use_idf=True)
train_data_features = vectorizer.fit_transform(train_data['clean'])

test_data_features = vectorizer.transform(test_data['clean'])

#SHAPE

train_data_features.shape, test_data_features.shape

#FUNCTION FOR MODEL TRAIN



((4928, 31661), (1233, 31661))

In [22]:
def train_n_test_classifier(clf, train_features, train_labels, test_features, test_labels,data):
    clf.fit(train_features, train_labels) # please learn patterns from the data

   
    print('-'*100+str(clf)[0:15])
    print("accuracy_score Score on training data:")
    print(clf.score(train_features, train_labels))
    
    
    print('_'*100)

    print("score on testing data:")
    
    pred_y = clf.predict(test_features)
    
    data['predict']=pred_y
 
    count=0
    
    print("accuracy_score Score on test data:")
    print(accuracy_score(test_labels, pred_y))
    
    print("f1_score  on test data:")
    print(f1_score(test_labels, pred_y, average='macro'))
    filename='result_'+str(clf)[0:15]+'.xlsx'
    test_data.to_excel(filename)
    
    
    sentiment_fit=clf.fit(train_features,train_labels)
    y_pred=sentiment_fit.predict(test_features)
    print(classification_report(test_labels,y_pred,target_names=('posative','negative','netural')))


In [23]:
#LOGASTIC REGRESSION



logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



# MultinomialNB

mnb = MultinomialNB()

train_n_test_classifier(mnb,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)
# S V M

svm = SVC(kernel='linear', probability=True, random_state=rand_seed)

train_n_test_classifier(svm,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


# MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)
train_n_test_classifier(mlp,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

# KNN

knn = KNeighborsClassifier(n_neighbors=3)

train_n_test_classifier(knn, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

#TREE
dtree = DecisionTreeClassifier(random_state=0)
train_n_test_classifier(dtree, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


estimator = []


estimator.append(('LR', LogisticRegression(solver ='lbfgs',  multi_class ='multinomial',  max_iter = 200)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('mnb',MultinomialNB()))
estimator.append(('mlp',MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=3)))     


vot_hard = VotingClassifier(estimators = estimator, voting ='hard')

train_n_test_classifier(vot_hard, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

train_n_test_classifier(vot_soft, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Gradientclf = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)

train_n_test_classifier(Gradientclf,train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

adaClf = AdaBoostClassifier(n_estimators=100, random_state=0)

train_n_test_classifier(adaClf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

rf = RandomForestClassifier(n_estimators=100, random_state=rand_seed)

train_n_test_classifier(rf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


----------------------------------------------------------------------------------------------------LogisticRegress
accuracy_score Score on training data:
0.927150974025974
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7575020275750203
f1_score  on test data:
0.7022195225687226
              precision    recall  f1-score   support

    posative       0.89      0.77      0.82       351
    negative       0.72      0.93      0.81       610
     netural       0.69      0.36      0.47       272

    accuracy                           0.76      1233
   macro avg       0.76      0.69      0.70      1233
weighted avg       0.76      0.76      0.74      1233

----------------------------------------------------------------------------------------------------MultinomialNB()
accuracy_score Score on training data:
0.885551948051948
____________________________________________________

Iteration 3, loss = 0.99737617
Iteration 4, loss = 0.87568724
Iteration 5, loss = 0.68323205
Iteration 6, loss = 0.50791346
Iteration 7, loss = 0.39025713
Iteration 8, loss = 0.28733067
Iteration 9, loss = 0.17992677
Iteration 10, loss = 0.10804972
Iteration 11, loss = 0.07399455
Iteration 12, loss = 0.06083911
Iteration 13, loss = 0.05372590
Iteration 14, loss = 0.05021615
Iteration 15, loss = 0.04697531
Iteration 16, loss = 0.04531875
Iteration 17, loss = 0.04351785
Iteration 18, loss = 0.04239885
Iteration 19, loss = 0.04082616
Iteration 20, loss = 0.03927261
Iteration 21, loss = 0.03915741
Iteration 22, loss = 0.03798657
Iteration 23, loss = 0.03632530
Iteration 24, loss = 0.03580720
Iteration 25, loss = 0.03477288
Iteration 26, loss = 0.03584506
Iteration 27, loss = 0.03453435
Iteration 28, loss = 0.03301035
Iteration 29, loss = 0.03317163
Iteration 30, loss = 0.03300771
Iteration 31, loss = 0.03244617
Iteration 32, loss = 0.03186429
Iteration 33, loss = 0.03220522
Iteration 34, l

----------------------------------------------------------------------------------------------------GradientBoostin
accuracy_score Score on training data:
0.8240665584415584
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7015409570154095
f1_score  on test data:
0.6643481222741139
              precision    recall  f1-score   support

    posative       0.75      0.72      0.74       351
    negative       0.72      0.80      0.76       610
     netural       0.57      0.44      0.50       272

    accuracy                           0.70      1233
   macro avg       0.68      0.66      0.66      1233
weighted avg       0.69      0.70      0.69      1233

----------------------------------------------------------------------------------------------------AdaBoostClassif
accuracy_score Score on training data:
0.726461038961039
___________________________________________________

In [25]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

# bow


vectorizer = CountVectorizer(ngram_range=(1, 2))
train_data_features = vectorizer.fit_transform(train_data['clean'])

test_data_features = vectorizer.transform(test_data['clean'])
#SHAPE

train_data_features.shape,  test_data_features.shape


((4928, 31661), (1233, 31661))

In [34]:
#LOGASTIC REGRESSION



logistic_reg = LogisticRegression(random_state=rand_seed)

train_n_test_classifier(logistic_reg, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)



# MultinomialNB

mnb = MultinomialNB()

train_n_test_classifier(mnb,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)
# S V M

svm = SVC(kernel='linear', probability=True, random_state=rand_seed)

train_n_test_classifier(svm,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


# MLPClassifier

mlp = MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)
train_n_test_classifier(mlp,  train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

# KNN

knn = KNeighborsClassifier(n_neighbors=3)

train_n_test_classifier(knn, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

#TREE
dtree = DecisionTreeClassifier(random_state=0)
train_n_test_classifier(dtree, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


estimator = []


estimator.append(('LR', LogisticRegression(solver ='lbfgs',  multi_class ='multinomial',  max_iter = 200)))
estimator.append(('SVC', SVC(gamma ='auto', probability = True)))
estimator.append(('DTC', DecisionTreeClassifier()))
estimator.append(('mnb',MultinomialNB()))
estimator.append(('mlp',MLPClassifier(hidden_layer_sizes=(20,20,20,20), verbose=True, tol=0.001, random_state=rand_seed)))
estimator.append(('knn',KNeighborsClassifier(n_neighbors=3)))     


vot_hard = VotingClassifier(estimators = estimator, voting ='hard')

train_n_test_classifier(vot_hard, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


vot_soft = VotingClassifier(estimators = estimator, voting ='soft')

train_n_test_classifier(vot_soft, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


Gradientclf = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)

train_n_test_classifier(Gradientclf,train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

adaClf = AdaBoostClassifier(n_estimators=100, random_state=0)

train_n_test_classifier(adaClf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)

rf = RandomForestClassifier(n_estimators=100, random_state=rand_seed)

train_n_test_classifier(rf, train_data_features, train_data[output],
                        test_data_features, test_data[output],test_data)


C:\Users\rabab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


----------------------------------------------------------------------------------------------------LogisticRegress
accuracy_score Score on training data:
0.981737012987013
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7599351175993512
f1_score  on test data:
0.7196339464699925


C:\Users\rabab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


              precision    recall  f1-score   support

    posative       0.85      0.81      0.83       351
    negative       0.77      0.86      0.81       610
     netural       0.58      0.46      0.51       272

    accuracy                           0.76      1233
   macro avg       0.74      0.71      0.72      1233
weighted avg       0.75      0.76      0.75      1233

----------------------------------------------------------------------------------------------------MultinomialNB()
accuracy_score Score on training data:
0.932426948051948
____________________________________________________________________________________________________
score on testing data:
accuracy_score Score on test data:
0.7639902676399026
f1_score  on test data:
0.6958005867378915
              precision    recall  f1-score   support

    posative       0.88      0.80      0.84       351
    negative       0.72      0.95      0.82       610
     netural       0.77      0.30      0.43       272

    acc

Iteration 15, loss = 0.03491901
Iteration 16, loss = 0.03456886
Iteration 17, loss = 0.03191503
Iteration 18, loss = 0.03211688
Iteration 19, loss = 0.03063822
Iteration 20, loss = 0.03002186
Iteration 21, loss = 0.03077645
Iteration 22, loss = 0.03133205
Iteration 23, loss = 0.02948324
Iteration 24, loss = 0.03149737
Iteration 25, loss = 0.03037153
Iteration 26, loss = 0.02891448
Iteration 27, loss = 0.02898964
Iteration 28, loss = 0.02956390
Iteration 29, loss = 0.02895896
Iteration 30, loss = 0.02809926
Training loss did not improve more than tol=0.001000 for 10 consecutive epochs. Stopping.
              precision    recall  f1-score   support

    posative       0.86      0.81      0.84       351
    negative       0.75      0.91      0.82       610
     netural       0.66      0.39      0.49       272

    accuracy                           0.77      1233
   macro avg       0.76      0.70      0.72      1233
weighted avg       0.76      0.77      0.75      1233

Iteration 1, loss